In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [2]:
import os

stream = os.popen('curl -fsS https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip -o /tmp/trainDevTestTrees_PTB.zip')
output = stream.read()
output

''

In [3]:
stream = os.popen('unzip -q -o -d /tmp /tmp/trainDevTestTrees_PTB.zip')
output = stream.read()
output

''

In [4]:
stream = os.popen('rm -f /tmp/trainDevTestTrees_PTB.zip')
output = stream.read()
output

''

In [5]:
def loadsst(path):
    xs = []
    ys = []
    file1 = open(path, 'r')
    Lines = file1.readlines()
    for line in Lines:
        soup = line.split()
        ys.append(int(soup[0].lstrip('(')))
        tokens = []
        for chunk in soup[2:]:
            if not chunk.endswith(")"):
                continue
            tokens.append(chunk.rstrip(')'))
        xs.append(tokens)
    return xs, ys

ssttrainxs, ssttrainys = loadsst("/tmp/trees/train.txt")
sstvalidxs, sstvalidys = loadsst("/tmp/trees/dev.txt")
ssttestxs, ssttestys = loadsst("/tmp/trees/test.txt") 


In [9]:
for i in range(32):
    print(ssttrainys[i], " ".join(ssttrainxs[i]))


3 the rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
4 the gorgeously elaborate continuation of `` the lord of the rings '' trilogy is so huge that a column of words can not adequately describe co-writer\/director peter jackson 's expanded vision of j.r.r. tolkien 's middle-earth .
3 singer\/composer bryan adams contributes a slew of songs -- a few potential hits , a few more simply intrusive to the story -- but the whole package certainly captures the intended , er , spirit of the piece .
2 you 'd think by now america would have had enough of plucky british eccentrics with hearts of gold .
3 yet the act is still charming here .
4 whether or not you 're enlightened by any of derrida 's lectures on `` the other '' and `` the self , '' derrida is an undeniably fascinating and playful fellow .
4 just the labour involved in creating the layered richness of the 

In [8]:
for i in range(len(ssttrainxs)):
  ssttrainxs[i] = [j.lower() for j in ssttrainxs[i]]   

In [10]:
character_set = set(" ".join([" ".join(ssttrainxs[i]) for i in range(len(ssttrainxs)) ]))

In [11]:
len(character_set)

67

In [12]:
int_to_char = {}
char_to_int = {}
c = 0
for i in character_set:
  int_to_char[c] = i
  char_to_int[i] = c
  c = c+1
print(int_to_char)
print(char_to_int)

{0: 'u', 1: 'i', 2: 'í', 3: 'n', 4: 'é', 5: 'ç', 6: '6', 7: 'á', 8: '/', 9: 'â', 10: 'p', 11: 'ñ', 12: '0', 13: 'd', 14: 'h', 15: '1', 16: 'v', 17: '.', 18: 'f', 19: 'ü', 20: 'k', 21: '2', 22: 'x', 23: '&', 24: '+', 25: '\\', 26: '9', 27: 'a', 28: 's', 29: '#', 30: ';', 31: 'l', 32: 'ô', 33: "'", 34: 'q', 35: 'e', 36: '=', 37: 'o', 38: 'b', 39: '8', 40: '4', 41: 'g', 42: 'r', 43: 'ó', 44: 'ï', 45: ',', 46: ' ', 47: 'ã', 48: '`', 49: '5', 50: 'm', 51: 't', 52: 'y', 53: '-', 54: 'z', 55: 'w', 56: '*', 57: '7', 58: ':', 59: 'è', 60: 'c', 61: '$', 62: '3', 63: '?', 64: 'æ', 65: '!', 66: 'j'}
{'u': 0, 'i': 1, 'í': 2, 'n': 3, 'é': 4, 'ç': 5, '6': 6, 'á': 7, '/': 8, 'â': 9, 'p': 10, 'ñ': 11, '0': 12, 'd': 13, 'h': 14, '1': 15, 'v': 16, '.': 17, 'f': 18, 'ü': 19, 'k': 20, '2': 21, 'x': 22, '&': 23, '+': 24, '\\': 25, '9': 26, 'a': 27, 's': 28, '#': 29, ';': 30, 'l': 31, 'ô': 32, "'": 33, 'q': 34, 'e': 35, '=': 36, 'o': 37, 'b': 38, '8': 39, '4': 40, 'g': 41, 'r': 42, 'ó': 43, 'ï': 44, ',': 45,

In [13]:
#creating a dataset of first n characters predicting n+1 th character.
n =100
train_list = [" ".join(ssttrainxs[i]) for i in range(len(ssttrainxs))] 
train_data = []
y_train = []
for sentence in train_list:
  for i in range(0,len(sentence)-n):
    train_data.append(sentence[i:i+n])
    y_train.append(sentence[i+n])

In [14]:
for i in range(50):
  print(train_data[i]," ",y_train[i])

the rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash    
he rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash    e
e rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash e   v
 rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash ev   e
rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash eve   n
ock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even    
ck is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even    g
k is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even g   r
 is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even gr   e
is destined to be the 21st century 's new `` conan '' a

In [15]:
def convert_char_to_int(train_data,y,char_to_int_dict):
  train_x = []
  train_y = []
  for i in range(len(train_data)):
    d = []
    for j in train_data[i]:
      num = char_to_int_dict[j]
      d.append(num)
    train_x.append(d)
    train_y.append(char_to_int_dict[y[i]])
  return train_x,train_y

In [16]:
train_x,train_y = convert_char_to_int(train_data,y_train,char_to_int)

In [17]:
import random
temp = list(zip(train_x,train_y))
random.shuffle(temp)
train_x,train_y = zip(*temp)
print(train_x[0:2],train_y[0:2])

([46, 55, 1, 51, 14, 46, 41, 37, 37, 13, 46, 1, 3, 51, 35, 3, 51, 1, 37, 3, 28, 46, 45, 46, 38, 0, 51, 46, 1, 3, 46, 14, 1, 28, 46, 54, 35, 27, 31, 46, 51, 37, 46, 28, 34, 0, 35, 35, 54, 35, 46, 51, 14, 35, 46, 27, 60, 51, 1, 37, 3, 46, 27, 3, 13, 46, 37, 0, 42, 46, 35, 50, 37, 51, 1, 37, 3, 28, 46, 1, 3, 51, 37, 46, 51, 14, 35, 46, 27, 31, 31, 53, 51, 37, 37, 53, 18, 27, 50, 1], [1, 35, 3, 60, 35, 46, 51, 14, 42, 37, 0, 41, 14, 46, 55, 37, 42, 13, 53, 37, 18, 53, 50, 37, 0, 51, 14, 46, 42, 35, 16, 1, 35, 55, 28, 46, 27, 3, 13, 46, 45, 46, 3, 37, 51, 46, 18, 27, 42, 46, 13, 37, 55, 3, 46, 51, 14, 35, 46, 31, 1, 3, 35, 46, 45, 46, 51, 37, 46, 18, 1, 3, 13, 46, 27, 46, 10, 31, 27, 60, 35, 46, 27, 50, 37, 3, 41, 46, 51, 14, 35, 46, 28, 51, 0, 13, 1, 37, 46, 33]) (31, 28)


In [18]:
import torch
from torch import nn
import argparse
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader



class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 7
        self.num_layers = 2
        self.input_size = n
        self.batch_size = 512
        n_vocab = len(character_set)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
            batch_first = True
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        #print("embed",embed.shape)
        output, state = self.lstm(embed, prev_state)
        #print("state",state[0].shape)
        #print("output",output.shape)
        logits = self.fc(output[:,-1,:])
        #print("logits", logits.shape)
        return logits, state

    def init_state(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.lstm_size),
                torch.zeros(self.num_layers, batch_size, self.lstm_size))
 

In [19]:

from torch.utils.data import DataLoader
def train(dataset, model, batch_size,max_epochs,sequence_length):
    model.train()

    dataloader = DataLoader(dataset, batch_size=batch_size,drop_last = True)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(max_epochs):
        state_h, state_c = model.init_state(batch_size)

        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()
            print("x",x.shape)
            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            print("state_h",state_h.shape)
            #print("y",y.shape)
            loss = loss_fn(y_pred, y)
            probs = torch.nn.functional.softmax(y_pred, dim=-1).detach().numpy()
            preds = np.argmax(probs,axis =1)
            acc = np.sum(preds==y.numpy())/(preds.shape[0])
            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item(),'batch_acc':acc })

In [20]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,train_x,train_y,character_set):
        self.train_x = train_x
        self.train_y = train_y
        self.character_set = character_set

    def __len__(self):
        return len(self.train_x) 

    def __getitem__(self, index):
        y_val = train_y[index]
        y_ohe = [0]*len(character_set)
        y_ohe[y_val] = 1
        return (
            torch.tensor(self.train_x[index]),
            torch.tensor(self.train_y[index]),
        )

In [21]:

training_data = Dataset(train_x,train_y,character_set)
model = Model(training_data)
train(training_data, model, batch_size = 512 , max_epochs = 5,sequence_length = 100)

Streaming output truncated to the last 5000 lines.
state_h torch.Size([2, 512, 128])
{'epoch': 0, 'batch': 198, 'loss': 2.2083888053894043, 'batch_acc': 0.341796875}
x torch.Size([512, 100])
state_h torch.Size([2, 512, 128])
{'epoch': 0, 'batch': 199, 'loss': 2.2604522705078125, 'batch_acc': 0.330078125}
x torch.Size([512, 100])
state_h torch.Size([2, 512, 128])
{'epoch': 0, 'batch': 200, 'loss': 2.167419910430908, 'batch_acc': 0.365234375}
x torch.Size([512, 100])
state_h torch.Size([2, 512, 128])
{'epoch': 0, 'batch': 201, 'loss': 2.2799997329711914, 'batch_acc': 0.30859375}
x torch.Size([512, 100])
state_h torch.Size([2, 512, 128])
{'epoch': 0, 'batch': 202, 'loss': 2.195927619934082, 'batch_acc': 0.34765625}
x torch.Size([512, 100])
state_h torch.Size([2, 512, 128])
{'epoch': 0, 'batch': 203, 'loss': 2.2178878784179688, 'batch_acc': 0.3125}
x torch.Size([512, 100])
state_h torch.Size([2, 512, 128])
{'epoch': 0, 'batch': 204, 'loss': 2.1136205196380615, 'batch_acc': 0.375}
x torch.S

In [28]:
def predict(model, text, next_chars=100,topk=3):
    model.eval()

    
    
    state_h, state_c = model.init_state(1)
    for i in range(0, next_chars):
        
        x = torch.tensor([[char_to_int[j] for j in text[i:].lower()]])

        y_pred, (state_h, state_c) = model(x, (state_h, state_c))
        #print(y_pred.shape)
        last_char_logits = y_pred[0]
        p = torch.nn.functional.softmax(last_char_logits, dim=-1).detach().numpy()
        #print(p.shape)
        #ind = np.argpartition(p, -1*topk)[-1*topk:]
        char_index = np.random.choice(len(last_char_logits),p=p)
        #char_index = np.argmax(p)
        text = text+int_to_char[char_index]

    return text

In [51]:
model

Model(
  (embedding): Embedding(94, 7)
  (lstm): LSTM(7, 128, dropout=0.2)
  (fc): Linear(in_features=128, out_features=93, bias=True)
)

In [29]:
test_text = "It 's been done before but never so vividly or with so "
complete_text = predict(model,test_text,1000)
complete_text

"It 's been done before but never so vividly or with so bale and who astoco-s on the bad the cinemation good little you seath to unuctumb dandy of their tod ... have to so good by proven moded . '' just ... up idathrentre . '' is quit does plaminac-all somethecelling to stars dreats to child ylu pasterity . '' to concess lead on though an goes as a vis and fasibrn ... for his dack deloaming moviegral centional band into aise of mouth 's spar impreost ... , though running to bey palrew , like sagule ... wond kbowfordy ago sersat and itsell compende other conquence to becartiarly audience lenge .ylorn . '' in enough by impoth through to querd witter . '' to poging ... to vinction ... the senner men work of its appense so kscevious cotosh conting asemphor least in many appoll and a because . '' -rrb- ... to lotner ... and confuntical that with to the carie . '' jood , but to that their modes , hock sumpry . '' it to the some fass fart , been does too as soulage of if them ... inconteriall

In [56]:
complete_text

"The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash  the t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t "

In [1]:
model.eval()

NameError: ignored

In [48]:
train(training_data, model, batch_size = 1024 , max_epochs = 5,sequence_length = 10)

{'epoch': 0, 'batch': 0, 'loss': 0.8239567279815674}
{'epoch': 0, 'batch': 1, 'loss': 0.733441174030304}
{'epoch': 0, 'batch': 2, 'loss': 0.39701682329177856}
{'epoch': 0, 'batch': 3, 'loss': 0.17971742153167725}
{'epoch': 0, 'batch': 4, 'loss': 0.1376793533563614}
{'epoch': 0, 'batch': 5, 'loss': 0.12338676303625107}
{'epoch': 0, 'batch': 6, 'loss': 0.07797113060951233}
{'epoch': 0, 'batch': 7, 'loss': 0.07060865312814713}
{'epoch': 0, 'batch': 8, 'loss': 0.0710889920592308}
{'epoch': 0, 'batch': 9, 'loss': 0.07016865164041519}
{'epoch': 0, 'batch': 10, 'loss': 0.19844573736190796}
{'epoch': 0, 'batch': 11, 'loss': 0.3606850504875183}
{'epoch': 0, 'batch': 12, 'loss': 0.427432656288147}
{'epoch': 0, 'batch': 13, 'loss': 0.3637229800224304}
{'epoch': 0, 'batch': 14, 'loss': 0.34360137581825256}
{'epoch': 0, 'batch': 15, 'loss': 0.3339104950428009}
{'epoch': 0, 'batch': 16, 'loss': 0.19630001485347748}
{'epoch': 0, 'batch': 17, 'loss': 0.08146701008081436}
{'epoch': 0, 'batch': 18, 'los

KeyboardInterrupt: ignored

In [40]:
a = torch.tensor([[7,5,9,6],[7,8,4,6]])
a.shape

torch.Size([2, 4])

In [42]:
word_embedding_layer = nn.Embedding(10, 3)
batch = word_embedding_layer(a)
batch_size = batch.shape[0]
(h, c) =  (torch.zeros(1, batch_size, 32), torch.zeros(1, batch_size, 32)) 
lstm  = nn.LSTM(3, 32, batch_first=True)

lstm_out, (h, c) = lstm(batch, (h, c))


In [43]:
lstm_out.shape

torch.Size([2, 4, 32])

In [44]:
h.shape

torch.Size([1, 2, 32])